In [ ]:
""" 

K-NEIBHORG CLASE multiclase MALWARE"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 
from sklearn import datasets, metrics
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report 
import os
import glob
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
import seaborn as sns

"""crea entorno grÃ¡fico"""
plt.rcParams['figure.figsize'] = (40, 50)
plt.style.use('ggplot')

# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
os.chdir("..\\Datasets")
os.getcwd()
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
df = pd.concat(list_data,ignore_index=True)

tipos=df.dtypes  #tipos de datos de campos

"""información datos"""
print(df.head(5))
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',df.columns)

"""Columnas, nulos y tipo de datos"""
print(df.info())

"""descripción estadística de los datos numéricos"""
print(df.describe())

"""correlación entre los datos"""
corr = df.set_index('Label').corr()
#sm.graphics.plot_corr(corr, xnames=list(corr.columns))
#plt.show()

df2=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3=df[['Label']]

"""convertir columna tipo dato str a int"""
df2['Label']=df3[['Label']]


"""
from sklearn.preprocessing import KBinsDiscretizer
bines = 4 # Elegir el número de bines
cabecera = list(df2) # Guardamos los nombres de las columnas.
ind = 0 # Contador para iterar por columnas.
while (ind < len(cabecera)):
    disc = df2.iloc[:,ind] 
    disc = disc.to_frame() 
    disc = KBinsDiscretizer(n_bins=bines, encode='ordinal',
                            strategy = "quantile").fit_transform(disc)
    df2[cabecera[ind]] = disc 
    ind = ind + 1
    del(disc)   
"""



"""desordena el dataset"""
"""desordena el dataset"""

df2=df2.sort_values('Fwd_Pkt_Len_Min')
df2 = df2[df2['Label'].isin([1,2,3,4])]



""" convertir valores número la clase"""
df2['Label'] = df2['Label'].replace(1,"Benigno")
df2['Label'] = df2['Label'].replace(2,"Adware")
df2['Label'] = df2['Label'].replace(3,"Scareware")
df2['Label'] = df2['Label'].replace(4,"Ransomware")
#print(df2.head(10))
#print(df2.groupby('Label').size())

dff=df2[:]
dff1 = dff[dff.Label == 'Ransomware']
dff = dff.drop(dff[dff['Label'] == 'Ransomware'].index)
a = dff1.iloc[0:41100,]
dff2 = pd.concat([dff,a])

dff=dff2[:]
dff1 = dff[dff.Label == 'Benigno']
dff = dff.drop(dff[dff['Label'] == 'Benigno'].index)
a = dff1.iloc[0:123500,]
dff2 = pd.concat([dff,a])

print(dff2.groupby('Label').size())

"""CREAMOS EL MODELO Y LO PROBAMOS CON LOS MISMOS DATOS SIN CROSS"""

  
X_train, X_test, Y_train, Y_test = train_test_split(dff2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
, dff2['Label'], test_size = 0.20, random_state=0)
# Instantiate the classifier
# training a KNN classifier 

from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier().fit(X_train, Y_train) 
accuracy = knn.score(X_train, Y_train) *100
print ("Precisión Entrenamiento " + str(accuracy))
  
# accuracy on X_test 
accuracy = knn.score(X_test, Y_test) *100
print ("Precisión Validado " + str(accuracy))
  
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(Y_test, knn_predictions) 
print(cm)
print("Métricas")
print(classification_report(Y_test, knn_predictions))


def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.predict_proba(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

plot_multiclass_roc(knn, X_test, Y_test, n_classes=4, figsize=(16, 10))

 
"""============================================="""
""" CROSS VALIDATION"""

X_trainCV, X_testCV, Y_trainCV, Y_testCV = train_test_split(dff2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],dff2['Label'], test_size=0.2, random_state=6) 

clfCV = KNeighborsClassifier().fit(X_train, Y_train) 

kf = KFold(n_splits=10)
score = clfCV.score(X_trainCV,Y_trainCV)
print("Metrica del modelo", score * 100)
scores = cross_val_score(clfCV, X_trainCV, Y_trainCV, cv=kf, scoring="accuracy")
print("Metricas cross_validation", scores * 100)
#print("Media de cross_validation", scores.mean() * 100)
preds = clfCV.predict(X_testCV)
score_pred = metrics.accuracy_score(Y_testCV, preds)
print("Metrica en Test", score_pred * 100)
conf_mat = confusion_matrix(Y_testCV, preds)
print(conf_mat)
print(classification_report(Y_testCV, preds))


#classifierCV=OneVsRestClassifier(LinearSVC(random_state=0))
#y_score = classifierCV.fit(X_train, Y_train).decision_function(X_test)

classifierCV=OneVsRestClassifier(KNeighborsClassifier())
y_score = classifierCV.fit(X_train, Y_train).predict_proba(X_test)



def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.predict_proba(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

plot_multiclass_roc(classifierCV, X_test, Y_test, n_classes=4, figsize=(16, 10))









"""
con
Label
Adware         40866
Benigno       137907
Ransomware     74888
Scareware      39672
dtype: int64
Precisión Entrenamiento 80.89795709646901
Precisión Validado 76.45354287759729
[[ 5235   455  2266   132]
 [  781 25338   944   493]
 [ 3283   669  9083  2115]
 [  281   425  1970  5197]]
Métricas
              precision    recall  f1-score   support
      Adware       0.55      0.65      0.59      8088
     Benigno       0.94      0.92      0.93     27556
  Ransomware       0.64      0.60      0.62     15150
   Scareware       0.65      0.66      0.66      7873
    accuracy                           0.76     58667
   macro avg       0.70      0.71      0.70     58667
weighted avg       0.77      0.76      0.77     58667
Distribution before resampling Counter({'Benigno': 110351, 'Ransomware': 59738, 'Adware': 32778, 'Scareware': 31799})
Distribution labels after resampling Counter({'Benigno': 110351, 'Ransomware': 110351, 'Adware': 110351, 'Scareware': 110351})
Precisión Entrenamiento 81.46550552328479
Precisión Validado 76.02911347094619
[[ 6069   349  1421   249]
 [  941 25191   795   629]
 [ 4467   375  7644  2664]
 [  427   222  1524  5700]]
Métricas
              precision    recall  f1-score   support
      Adware       0.51      0.75      0.61      8088
     Benigno       0.96      0.91      0.94     27556
  Ransomware       0.67      0.50      0.58     15150
   Scareware       0.62      0.72      0.67      7873
    accuracy                           0.76     58667
   macro avg       0.69      0.72      0.70     58667
weighted avg       0.78      0.76      0.76     58667
sin
Label
Adware         40866
Benigno       137907
Ransomware     74888
Scareware      39672
dtype: int64
Precisión Entrenamiento 73.58586245983653
Precisión Validado 61.34624235089573
[[ 2237  1207  3861   919]
 [  886 24679  1476   421]
 [ 3657  1905  7472  1923]
 [ 1804  1038  3580  1602]]
Métricas
              precision    recall  f1-score   support
      Adware       0.26      0.27      0.27      8224
     Benigno       0.86      0.90      0.88     27462
  Ransomware       0.46      0.50      0.48     14957
   Scareware       0.33      0.20      0.25      8024
    accuracy                           0.61     58667
   macro avg       0.48      0.47      0.47     58667
weighted avg       0.60      0.61      0.60     58667
Distribution before resampling Counter({'Benigno': 110445, 'Ransomware': 59931, 'Adware': 32642, 'Scareware': 31648})
Distribution labels after resampling Counter({'Benigno': 110445, 'Scareware': 110445, 'Ransomware': 110445, 'Adware': 110445})
Precisión Entrenamiento 85.89094119244874
Precisión Validado 58.74682530212897
[[ 2969   560  2668  2027]
 [ 1647 23169  1307  1339]
 [ 4836   767  5270  4084]
 [ 2265   422  2280  3057]]
Métricas
              precision    recall  f1-score   support
      Adware       0.25      0.36      0.30      8224
     Benigno       0.93      0.84      0.88     27462
  Ransomware       0.46      0.35      0.40     14957
   Scareware       0.29      0.38      0.33      8024
    accuracy                           0.59     58667
   macro avg       0.48      0.48      0.48     58667
weighted avg       0.63      0.59      0.60     58667
"""

['benign-2c .csv', 'malware.csv']
   Flow_Duration  Tot_Fwd_Pkts  Tot_Bwd_Pkts  TotLen_Fwd_Pkts  \
0           5870             2             0              813   
1           1499             2             0              128   
2             16             0             2                0   
3          46435             1             1               79   
4              0             1             0              356   

   TotLen_Bwd_Pkts  Fwd_Pkt_Len_Min  Bwd_Pkt_Len_Min  Fwd_Pkt_Len_Max  \
0                0               83             -1.0              730   
1                0               64             -1.0               64   
2              159               -1             52.0               -1   
3               95               79             95.0               79   
4                0              356             -1.0              356   

   Bwd_Pkt_Len_Max  Fwd_Pkt_Len_Mean  Bwd_Pkt_Len_Mean  Fwd_Pkt_Len_Std  \
0               -1             406.5               0.0       